# Import Prometheus Snapshots onto a Fabric slice

### Prerequisite
- A running fabric slice (For example: [Create single node backup Slice](./create_single_node_backup.ipynb))
- A Prometheus snapshot tar file to import

#### General Imports

In [ ]:
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager
try:
    fablib = fablib_manager()                
except Exception as e:
    print(f"Exception: {e}")

#### Get your slice information
*Set slice_name and node_name below before running*

In [ ]:
slice_name='backup-testing'
node_name='data-backup'


slice = fablib.get_slice(name=slice_name)
slice.show();
node = slice.get_node(name=node_name)
node.show();

#### Import ELK import/export service tools

In [ ]:
from mflib.data_transfer import PrometheusImporter
prom_tools = PrometheusImporter(slice_name=slice_name, node_name=node_name)

---
## Set up your Docker-Compose App (Prometheus and Grafana)

#### Ensure Docker and Docker-Compose are installed on the node

In [ ]:
prom_tools.install_docker()

#### Set up NAT64 Server
We need to do this step so that we can clone the Github repo with our docker app.

In [ ]:
prom_tools.setup_nat64(node_name)

### Create docker container and import your data

#### Clone Docker App Repository
Clones https://github.com/jackhancockuky/data-import-containers.git to your docker container.
This repository contains a docker-compose app which will allow you to automatically set up Elastic Search and Kibana on your node.

In [ ]:
prom_tools.clone_repository()

---
### Upload snapshot tar file to your node

#### Option 1: Upload to node from local device with SCP

##### Prerequisite
- For this step, you need your local ssh environment configured ([Configure Environment](./configure.ipynb))

##### Example SCP (Secure Copy) command to download snapshot file from VM

scp -F ~/.ssh/fabric/ssh_config -i ~/.ssh/fabric/slice_key ubuntu@\[2620:0:c80:1001:f816:3eff:fe81:80cc]:/home/mfuser/services/elk/files/snapshots/snapshot.tar .

##### Fill in the variables below then run to generate your custom SCP command

In [ ]:
# The ssh config file location on your local device
ssh_config = "~/.ssh/fabric/ssh_config"

# The fabric slice private key file on your local device
private_key = "~/.ssh/fabric/slice_key"

# The Nodes username
username = node.get_username()

# The Nodes IP Address
ip = node.get_management_ip()

# Destination to store snapshot file on your local device
local_file_location = "/Users/jackhancock/Desktop/"

# Snapshot file name
snapshot_file_name = "20230905T150325Z-09f5419c7b3c5f90.tar"

# Create and print command
scp_command = f"scp -F {ssh_config} -i {private_key} {local_file_location}{snapshot_file_name} {username}@\[{ip}]:/tmp/{snapshot_file_name}"
print(scp_command)

##### More info
Fabric guide for transferring data directly from VM to local machine:
https://learn.fabric-testbed.net/knowledge-base/transferring-data-to-and-from-your-vms/

#### Run the import snapshot script

In [ ]:
prom_tools.import_snapshot("20230905T150325Z-09f5419c7b3c5f90.tar")

#### Start the docker compose app

In [ ]:
prom_tools.start_docker()

### Your data is now imported on the node.

----
### View data from Grafana

#### On your local terminal create a tunnel with:

'ssh -L 10030:localhost:5601 -F ssh_config -i slice_key ubuntu@{node_ip}'

#### Then go to http://localhost:10030
----

----
### Query Prometheus Metrics

In [ ]:
prom_tools.view_snapshot("imported_repository", "snapshot_test")

In [ ]:
prom_tools.view_indices()

### To reuse slice with new data (if you are done then just delete the slice)
#### Take down container and remove data

In [ ]:
#prom_tools.stop_docker()
#prom_tools.remove_data()

#### Rerun steps starting from *Create Docker container and import your data*